<a href="https://colab.research.google.com/github/shionguha/inf2178-expdesignfordatascience-w23/blob/main/Logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [77]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import numpy as np
pd.options.display.max_columns = None #show all columns in data

# Load dataset

In [78]:
df = pd.read_csv("student-por.csv")

In [79]:
df.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,reason,guardian,traveltime,studytime,failures,schoolsup,famsup,paid,activities,nursery,higher,internet,romantic,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,course,mother,2,2,0,yes,no,no,no,yes,yes,no,no,4,3,4,1,1,3,4,0,11,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,course,father,1,2,0,no,yes,no,no,no,yes,yes,no,5,3,3,1,1,3,2,9,11,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,other,mother,1,2,0,yes,no,no,no,yes,yes,yes,no,4,3,2,2,3,3,6,12,13,12
3,GP,F,15,U,GT3,T,4,2,health,services,home,mother,1,3,0,no,yes,no,yes,yes,yes,yes,yes,3,2,2,1,1,5,0,14,14,14
4,GP,F,16,U,GT3,T,3,3,other,other,home,father,1,2,0,no,yes,no,no,yes,yes,no,no,4,3,2,1,2,5,0,11,13,13


# Select features of interest

In [80]:
#I select some random numerical features. Note, you will want to one hot encode non-numerical features 
x = df[['age', 'traveltime', 'studytime', 'health', 'absences', 'G1', 'paid']].copy()
x['paid'] = pd.get_dummies(df.paid, drop_first=True)

# Select the response column 
y = pd.get_dummies(df.romantic, drop_first=True)

In [81]:
x.groupby('paid').size()

paid
0    610
1     39
dtype: int64

In [82]:
x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 649 entries, 0 to 648
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   age         649 non-null    int64
 1   traveltime  649 non-null    int64
 2   studytime   649 non-null    int64
 3   health      649 non-null    int64
 4   absences    649 non-null    int64
 5   G1          649 non-null    int64
 6   paid        649 non-null    uint8
dtypes: int64(6), uint8(1)
memory usage: 31.2 KB


In [83]:
y['yes'].unique()

array([0, 1], dtype=uint8)

# Train test split 

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html 

In [84]:
# Split these data into training and testing datasets
#set random seed for replicability 
#you may want to consider stratifying your train test split (creates splits by preserving the same percentage for each target class as in the complete set.)
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state=123)

# Running logistic regression 

In [85]:
y_train

,yes
243,1
234,0
527,0
177,0
226,0
...,...
98,0
322,0
382,0
365,1


In [86]:
#building the model with statsmodels
import statsmodels.formula.api as smf

# Create the formula string 
all_columns = ' + '.join(x)
formula = "yes ~ " + all_columns 
print("Formula: ", formula, "\n")

# Put the training predictors and responses into one DataFrame to be input into the model
trainingdata = pd.concat([x_train,y_train], axis = 1)

# Build the model
log_reg_1 = smf.logit(formula, data=trainingdata).fit()

Formula:  yes ~ age + traveltime + studytime + health + absences + G1 + paid 

Optimization terminated successfully.
         Current function value: 0.627724
         Iterations 5


In [87]:
log_reg_1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    yes   No. Observations:                  519
Model:                          Logit   Df Residuals:                      511
Method:                           MLE   Df Model:                            7
Date:                Mon, 27 Mar 2023   Pseudo R-squ.:                 0.03109
Time:                        18:59:32   Log-Likelihood:                -325.79
converged:                       True   LL-Null:                       -336.24
Covariance Type:            nonrobust   LLR p-value:                  0.003914
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -4.9681      1.522     -3.264      0.001      -7.951      -1.985
age            0.2717      0.080      3.390      0.001       0.115       0.429
traveltime    -0.0327      0.128     -0.254      0.799      -0.284       0.219
studytime      0.1587      0.116      1.373      0.170      -0.068       0.385
health        -0.0171      0.066     -0.259      0.795      -0.146       0.112
absences       0.0292      0.021      1.391      0.164      -0.012       0.070
G1            -0.0464      0.037     -1.268      0.205      -0.118       0.025
paid          -0.0622      0.380     -0.164      0.870      -0.807       0.682
==============================================================================
"""

In [88]:
#another way 
reg = smf.logit('yes ~ age + traveltime + studytime + health + absences + G1 +paid', data=trainingdata).fit()
print(reg.summary())

Optimization terminated successfully.
         Current function value: 0.627724
         Iterations 5
                           Logit Regression Results                           
Dep. Variable:                    yes   No. Observations:                  519
Model:                          Logit   Df Residuals:                      511
Method:                           MLE   Df Model:                            7
Date:                Mon, 27 Mar 2023   Pseudo R-squ.:                 0.03109
Time:                        18:59:32   Log-Likelihood:                -325.79
converged:                       True   LL-Null:                       -336.24
Covariance Type:            nonrobust   LLR p-value:                  0.003914
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -4.9681      1.522     -3.264      0.001      -7.951      -1.985
age            0.2717      0.

**Interpretation of the above coefficients (not preferred):** Increasing the age of the student by one unit increases in predicted log of the odds of a student being in a romantic relationship by 0.2717 holding other features constant. However, the feature is not statistically significant. 

Note if the feature is categorical and has a reference level (e.g. say if the student has a paid job or not) then you would interpret things as:
Compared with students who did not have a paid job, the log odds of getting the student being in a relationship who have a paid job are 0.0622 lower, controlling for other features. Again this feature is not statistically significant. 

**Now, the above is not too interpretable. You can get the odds ratio which makes the features more interpretable. Take the exponent of the coefficients.**  

$\text{Odds ratio} =\frac{\text{Odds after a unit change in the predictor}}{\text{Original odds}}$. 


If the odds are the same across groups, the odds ratio (OR) will be 1.0. If not, the OR will be larger or smaller than one. OR>1, if predictor increases then probability of outcome occuring increases. When OR<1, if predictor increases, then probability of outcome occuring decreases

In [89]:
#getting odds ratio 
np.exp(reg.params)

Intercept     0.006956
age           1.312174
traveltime    0.967871
studytime     1.171990
health        0.983078
absences      1.029625
G1            0.954700
paid          0.939663
dtype: float64

**Interpretation (prefer this):** We perform a logistic regression to examine the effects of age, travel time, study time, health, absences, G1 grades, paid job, on the likelihood that students are in a relationship. None of the features are statistically significant.   

From here you normally report the features that are statistically significant. E.g. if age was statistically significant one would say: Increasing age was associated with an increased likelihood of a student being in a romantic relationship. The odds ratio is approximately 1.31 For each additional age increase for the student, the odds that the student is in a romantic relationship increases by about 1.31 times.    

**For categorical features, you want to know what the reference category is to make a correct interpretation of your results.**



In [90]:
params = reg.params
conf = reg.conf_int()
conf['OR'] = params
conf.columns = ["Lower CI", "Upper CI", "OR"]
np.exp(conf)

,Lower CI,Upper CI,OR
Intercept,0.000352,0.137359,0.006956
age,1.121447,1.535338,1.312174
traveltime,0.752502,1.244879,0.967871
studytime,0.934430,1.469945,1.171990
health,0.864063,1.118486,0.983078
absences,0.988122,1.072872,1.029625
G1,0.888707,1.025593,0.954700
paid,0.446335,1.978259,0.939663


# assess the model 

In [91]:
from sklearn.metrics import confusion_matrix, accuracy_score
# Predict responses 
pred_1 = log_reg_1.predict(x_test)
# round() rounds to nearest integer;
# 0.5 rounds to 0; 0.501 rounds to 1
prediction_1 = list(map(round, pred_1))

In [92]:
# Accuracy score
print('\nTest accuracy = ', accuracy_score(y_test, prediction_1))


Test accuracy =  0.5846153846153846


#Make sure you talk about the confusion matrix for your logistic regression model 

In [93]:
# Confusion matrix
#https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html
#C0,0 is true negative, C1,0 is false negative, C1,1 is true positive, C0,1 false positive 

cm = confusion_matrix(y_test, prediction_1) 
print ("\nConfusion Matrix : \n", cm) 


Confusion Matrix : 
 [[70  3]
 [51  6]]
